**Define Output Dirs**

In [2]:
map_output_dir = '/Users/jt041/repos/prism/testing_outputs/test_maps/'
meta_output_dir = '/Users/jt041/repos/prism/testing_outputs/'
results_output_prefix = '/Users/jt041/repos/prism/testing_outputs/prism_output/prism'

**Create Synthetic NIfTI Data**

In [ ]:
from prism.datasets import generate_null_brain_map, fetch_mni152_2mm_mask_img
import numpy as np
from tqdm import tqdm
from nilearn.maskers import NiftiMasker
import os
import pandas as pd

os.makedirs(map_output_dir, exist_ok=True)
os.makedirs(meta_output_dir, exist_ok=True)

# Create random maps
mask_img = fetch_mni152_2mm_mask_img()
n_maps = 100
imgs = []
img_paths = []
for i in tqdm(range(n_maps)):
    if i < 11:
        random_state = 42
    elif i > (n_maps - 11):
        random_state = 37
    else:
        random_state = i
    imgs.append(generate_null_brain_map(mask_img, random_state=random_state))
    imgs[i].to_filename(os.path.join(map_output_dir, f'null_brain_map_{i+1}.nii.gz'))
    img_paths.append(os.path.join(map_output_dir, f'null_brain_map_{i+1}.nii.gz'))

# Save paths and concatenate images
np.random.seed(42)
masker = NiftiMasker(mask_img=mask_img)
data = masker.fit_transform(imgs)
concatenated_img = masker.inverse_transform(data)
concatenated_img.to_filename(os.path.join(meta_output_dir, 'null_brain_maps_concatenated.nii'))
pd.DataFrame(img_paths).to_csv(os.path.join(meta_output_dir, 'null_brain_map_paths.csv'), index=False, header=False)

# Define design matrix
design_col1 = np.hstack([np.ones(10)+0.75, np.random.randn(n_maps-10)])
design_col1 += np.random.randn(n_maps) * 0.01
design_col2 = np.hstack([np.random.randn(n_maps-10), np.ones(10)-1.75])
design_col2 += np.random.randn(n_maps) * 0.01
design_col3 = np.random.randn(n_maps) * 2
intercept = np.ones(n_maps)
design = np.vstack([design_col1, design_col2, design_col3, intercept]).T
design = design.astype(np.float16)

# Define T and F contrasts
contrast_r1 = np.atleast_2d(np.array([1, 0, 0, 0]))
contrast_r2 = np.atleast_2d(np.array([0, 1, 0, 0]))
contrast = np.vstack([contrast_r1, contrast_r2])
f_contrast_indices = [1,1]

# Define exchangeability blocks
exchangeability_blocks = np.hstack([np.ones(np.floor(n_maps / 2).astype(int)) * i+1 for i in range(2)])
if n_maps % 2 != 0:
    exchangeability_blocks = np.hstack([exchangeability_blocks, np.ones(n_maps % 2) * 2])


# Save design and contrast as csvs and npys
np.save(os.path.join(meta_output_dir, 'design.npy'), design)
np.save(os.path.join(meta_output_dir, 'contrast.npy'), contrast)
np.save(os.path.join(meta_output_dir, 'exchangeability_blocks.npy'), exchangeability_blocks)
np.save(os.path.join(meta_output_dir, 'f_contrast_indices.npy'), f_contrast_indices)
pd.DataFrame(design).to_csv(os.path.join(meta_output_dir, 'design.csv'), index=False, header=False)
pd.DataFrame(contrast).to_csv(os.path.join(meta_output_dir, 'contrast.csv'), index=False, header=False)
pd.DataFrame(exchangeability_blocks).to_csv(os.path.join(meta_output_dir, 'exchangeability_blocks.csv'), index=False, header=False)
pd.DataFrame(np.atleast_2d(f_contrast_indices)).to_csv(os.path.join(meta_output_dir, 'f_contrast_indices.csv'), index=False, header=False)


**Create System Commands for both Prism and PALM**

In [4]:
from IPython.display import HTML, display
import os

mask_path = '/Users/jt041/repos/prism/prism/data/MNI152_T1_2mm_brain_mask.nii.gz'

prism_cmd = f"""
prism
-i {os.path.join(meta_output_dir, 'null_brain_maps_concatenated.nii')}
-m {mask_path}
-d {os.path.join(meta_output_dir, 'design.csv')}
-t {os.path.join(meta_output_dir, 'contrast.csv')}
-f {os.path.join(meta_output_dir, 'f_contrast_indices.csv')}
-n 50
-o {results_output_prefix}
-accel tail
-save1-p
"""
prism_cmd = prism_cmd.replace('\n', ' ').strip()

palm_cmd = prism_cmd.replace('prism', 'palm').replace('/palm/', '/prism/')


# build the HTML string
html = f"""
<div style="margin-bottom:1em;">
  <pre id="prism-cmd" style="border:1px solid #ccc; padding:8px; white-space: pre-wrap;">{prism_cmd}</pre>
  <button onclick="
    navigator.clipboard.writeText(document.getElementById('prism-cmd').innerText)
      .then(() => this.textContent='Copied!')
      .catch(() => this.textContent='Failed');
  ">Copy Prism Command</button>
</div>
<div style="margin-bottom:1em;">
  <pre id="palm-cmd" style="border:1px solid #ccc; padding:8px; white-space: pre-wrap;">{palm_cmd}</pre>
  <button onclick="
    navigator.clipboard.writeText(document.getElementById('palm-cmd').innerText)
      .then(() => this.textContent='Copied!')
      .catch(() => this.textContent='Failed');
  ">Copy PALM Command</button>
</div>
"""

# display it
display(HTML(html))

**Run sys commands**

In [ ]:
%%bash
prism -i /data/nimlab/USERS/joseph/external_python_modules/prism/testing_outputs/null_brain_maps_concatenated.nii -m /data/nimlab/software/miniforge_2024-08-31/packages/envs/nimlab_dev/lib/python3.11/site-packages/nimlab/data/MNI152_T1_2mm_brain_mask.nii -d /data/nimlab/USERS/joseph/external_python_modules/prism/testing_outputs/design.csv -t /data/nimlab/USERS/joseph/external_python_modules/prism/testing_outputs/contrast_1d.csv -n 150 -o /data/nimlab/USERS/joseph/external_python_modules/prism/testing_outputs/prism_output/prismacceltail -accel tail -save1-p